# Compression on general topologies

In this notebook we will extend the ideas discussed in [notebook 4](4.Gate-decomposition-and-tensor-network-compression.ipynb) to arbitrary topologies. We develop and demonstrate these ideas using 2D Random Quantum Circuits (RQCs). 

First as always we import the PicoQuant package and also the Plots package for producing plots.

In [ ]:
using PicoQuant
import Plots

## Create a random quantum circuit

We create a random quantum circuit similar to those used in the quantum advantage experiments. These circuits are defined on 2D grids of qubits and feature:
- An initial layer of Hadamard gates
- Layers of entangling gates (in this case controlled Z gates) applied on neightboring qubits in a pattern which takes 8 layers to cover to all pairs
- Layers of single single qubit gates chosen at random from a set of T, sqrt(X) and sqrt(Y) gates which are applied to qubits to which entangling gates were applied in the last round. These are applied such that the same gate is not applied to the same qubit twice in a row.

We now create such a circuit for a 3x3 grid with depth 16 which corresponds to two complete coverings of pairs of neighbouring qubits.

In [ ]:
rows = 3
cols = 3
depth = 32
seed = 42 # we set the seed to ensure reproducibility while debuggin
          # replace this with `nothing` to vary the circuit
circ = create_RQC(rows, cols, depth, seed) 
circ.draw()

## Contract with full wavefunction approach

We first contract this circuit using the full wave-function approach to get the exact wave-function output which will be used as a reference.

In [ ]:
tn = convert_qiskit_circ_to_network(circ)
add_input!(tn, "0"^(rows * cols))
output_node = full_wavefunction_contraction!(tn, "vector")
ψ = load_tensor_data(tn, output_node)
println("State norm: $(abs(ψ' * ψ)^2)")

## MPS contracton approach

We contract using the MPS approach with no cutoff applied to the bond dimension. This should give the exact state and also show the bond dimension that is required to represent this state exactly.

In [ ]:
tn = convert_qiskit_circ_to_network(circ, decompose=true, transpile=true)
add_input!(tn, "0"^(rows * cols))
@time mps_nodes = contract_mps_tensor_network_circuit!(tn)
@show [getnode(tn, x).dims for x in mps_nodes]
@show [prod(getnode(tn, x).dims) for x in mps_nodes]
ψ_mps = calculate_mps_amplitudes!(tn, mps_nodes)
println("State norm: $(abs(ψ_mps' * ψ_mps)^2)")
println("Overlap with exact from full wf approach: $(abs(ψ' * ψ_mps)^2)")

We see from this that the bond dimension in the centre of the chain has grown to 16. Next we apply a cut off on the maximum dimension of these bonds and compare the outputs.

In [ ]:
tn = convert_qiskit_circ_to_network(circ, decompose=true, transpile=true)
add_input!(tn, "0"^(rows * cols))
mps_nodes = contract_mps_tensor_network_circuit!(tn, max_rank=14)
@show [getnode(tn, x).dims for x in mps_nodes]
ψ_mps′ = calculate_mps_amplitudes!(tn, mps_nodes);
println("State norm: $(abs(ψ_mps′' * ψ_mps′)^2)")
println("Overlap : $(abs(ψ' * ψ_mps′)^2)")

println("\nWe now normalise the output state first and check the overlap")
ψ_mps_normed′ = ψ_mps′ / sqrt(abs(ψ_mps′' * ψ_mps′))
println("State norm: $(abs(ψ_mps_normed′' * ψ_mps_normed′)^2)")
println("Overlap : $(abs(ψ' * ψ_mps_normed′)^2)")

Here we see that by truncating the bond dimension to 14 we get and approximation of the exact output state with smaller overlap.

## PEPs contraction approach

Instead of using an MPS to represent the state we attempt to use a PEPS state which better matches the topology of the input circuit. The first step here is to be able to generate a coupling map which returns the sparse adjacency matrix representation of the lattice. This coupling map can be used to transpile the circuit so that entangling gates are only applied between neighbouring qubits.

In [ ]:
"""
generate_2d_coupling_map(rows::Integer, cols::Integer)

Generate a coupling map for a grid
"""
function generate_2d_coupling_map(rows::Integer, cols::Integer)
    coupling_map = Array{Array{Int64, 1}}(undef, 0)
    for i in 1:rows
        for j in 1:cols
            idx = (i-1)*rows + j
            if i < rows
                push!(coupling_map, [idx - 1, idx+rows - 1])
            end
            if j < cols
                push!(coupling_map, [idx - 1, idx])
            end
        end
    end
    coupling_map
end

In [ ]:
coupling_map = generate_2d_coupling_map(rows, cols)

We convert the RQC to a tensor network circuit using this coupling map and compare the resulting tensor network to what results when using the default coupling map (corresonding to a 1D chain of tensors called (an MPS)).

In [ ]:
seed = 42 # we set the seed to ensure reproducibility while debuggin
          # replace this with `nothing` to vary the circuit
circ = create_RQC(rows, cols, depth, seed) 

In [ ]:
tn = convert_qiskit_circ_to_network(circ, decompose=true, transpile=true, couplings=coupling_map)
add_input!(tn, "0"^(rows*cols))
plot(tn)

From this we can see that for the MPS layout, the resulting circuit has many more gates which must be added to satisfy the constraint that only neighbouring qubits can have entangling gates applied to them.

Next we contract this network in order without any compression and look at the resulting bond dimensions.

In [ ]:
tn = convert_qiskit_circ_to_network(circ, decompose=true, transpile=true, couplings=coupling_map)
add_input!(tn, "0"^(rows*cols))
@time network_nodes = contract_tensor_network_circuit_with_compression!(tn)
[getnode(tn, x).dims for x in network_nodes]

To get all the amplitudes of the PEPS state it is necessary to contract the resulting PEPS network to a single tensor. This can be very computationally expensive and is very sensitive to the contracting ordering used. For the 3x3 case we are using here we can get away with using the `calculate_mps_amplitudes!` along with the array of node names returned from the contraction function. For larger PEPS states however, it will be necessary to take greater care to choose a good ordering.

In [ ]:
ψ_peps = calculate_mps_amplitudes!(tn, network_nodes);
println("State norm: $(abs(ψ_peps' * ψ_peps)^2)")
println("Overlap with exact from full wf approach: $(abs(ψ' * ψ_peps)^2)")

Next we apply a cut-off to the bond dimension and see the effect on the fidelity of the output state.

In [ ]:
tn = convert_qiskit_circ_to_network(circ, decompose=true, transpile=true, couplings=coupling_map)
add_input!(tn, "0"^(rows*cols))
network_nodes = contract_tensor_network_circuit_with_compression!(tn, max_rank=15)
@show [getnode(tn, x).dims for x in network_nodes]
ψ_peps′ = calculate_mps_amplitudes!(tn, network_nodes)
println("State norm: $(abs(ψ_peps′' * ψ_peps′)^2)")
println("Overlap with exact from full wf approach: $(abs(ψ' * ψ_peps′)^2)")

We see that this had a large impact on the fidelity of the output state, which we can improve somewhat by normalising the state.

In [ ]:
ψ_peps_normed′ = ψ_peps′ / sqrt(abs(ψ_peps′' * ψ_peps′))
println("State norm: $(abs(ψ_peps_normed′' * ψ_peps_normed′)^2)")
println("Overlap with exact from full wf approach: $(abs(ψ' * ψ_peps_normed′)^2)")

We can plot some of the individual amplitudees to get an idea of how they compare for the different approximations.

In [ ]:
Plots.plot(abs.(ψ[1:100]).^2, label="Exact")
Plots.plot!(abs.(ψ_mps_normed′[1:100]).^2, label="Truncated MPS")
Plots.plot!(abs.(ψ_peps_normed′[1:100]).^2, label="Truncated PEPS", ylabel="prob. amplitude", xlabel="index")

In this notebook we showed the contraction of a network using the PEPS representation for the input state. This exactly reproduced the output state when no cut-off was applied and gave an approximation with non zero overlap when a cut-off was used.

- Some more work is required to select appropriate orderings for contracting the full output PEPS state efficently and for being able to calculate particular amplitudes similar to how the MPSState type works. 
- From the crude timing above, it appears that the PEPS approach takes a similar amount of time as the MPS approach but has fewer allocations and uses less memory. Further work is required to characterise the scaling behaviour for larger and deeper RQCs.